In [6]:
import pywren
import pickle 
import time
import boto3
import numpy as np

def latency(n):
    s3_client = boto3.client('s3')
    bucket_name = 'latency-kostis'
    
    random_num = np.random.rand(1, n) #generate n floats (each becomes 20B after serializing)
    rand_serial = pickle.dumps(random_num)
    t0 = time.time()
    for i in range(100):
        s3_client.put_object(
            Bucket = bucket_name,
            Body = rand_serial,
            Key = 'tmp'+str(n)+'/'+str(i)
        )   
    t1 = time.time()
    for i in range(100):
        s3_client.get_object(Bucket=bucket_name, Key='tmp'+str(n)+'/'+str(i))['Body'].read()
    t2 = time.time()
    return (n, (t1-t0)*10, (t2-t1)*10) # time in ms 

In [7]:
wrenexec = pywren.default_executor()

In [8]:
args_list = [1] * 1
futures = wrenexec.map(latency, args_list)
result = [0, 0]
result_full = [[], []]
for future in futures:
    res = future.result()
    result[0] += res[1]
    result_full[0].append(res[1])
    result[1] += res[2]
    result_full[1].append(res[2])
result[0] /= len(futures)
result[1] /= len(futures)
result_write = np.array(result_full[0])
result_read = np.array(result_full[1])
print [np.percentile(result_write, 100), np.percentile(result_read, 100)]
print result

[24.537880420684814, 11.810991764068604]
[24.537880420684814, 11.810991764068604]


In [15]:
args_list = [1] * 10
futures = wrenexec.map(latency, args_list)
result = [0, 0]
result_full = [[], []]
for future in futures:
    res = future.result()
    result[0] += res[1]
    result_full[0].append(res[1])
    result[1] += res[2]
    result_full[1].append(res[2])
result[0] /= len(futures)
result[1] /= len(futures)
result_write = np.array(result_full[0])
result_read = np.array(result_full[1])
print [np.percentile(result_write, 100), np.percentile(result_read, 100)]
print result

[26.474411487579346, 16.700279712677002]
[24.388997316360474, 14.271677017211914]


In [13]:
args_list = [1] * 100
futures = wrenexec.map(latency, args_list)
result = [0, 0]
result_full = [[], []]
for future in futures:
    res = future.result()
    result[0] += res[1]
    result_full[0].append(res[1])
    result[1] += res[2]
    result_full[1].append(res[2])
result[0] /= len(futures)
result[1] /= len(futures)
result_write = np.array(result_full[0])
result_read = np.array(result_full[1])
print [np.percentile(result_write, 100), np.percentile(result_read, 100)]
print result

[39.418289661407471, 17.398779392242432]
[27.095448565483093, 13.480706119537354]


In [12]:
args_list = [1] * 1000
futures = wrenexec.map(latency, args_list)
result = [0, 0]
result_full = [[], []]
for future in futures:
    res = future.result()
    result[0] += res[1]
    result_full[0].append(res[1])
    result[1] += res[2]
    result_full[1].append(res[2])
result[0] /= len(futures)
result[1] /= len(futures)
result_write = np.array(result_full[0])
result_read = np.array(result_full[1])
print [np.percentile(result_write, 100), np.percentile(result_read, 100)]
print result

The pickling exception: ('__init__() takes exactly 3 arguments (2 given)', <class 'botocore.exceptions.ClientError'>, (u'An error occurred (SlowDown) when calling the PutObject operation (reached max retries: 4): Please reduce your request rate.',))


Exception: An error occurred (SlowDown) when calling the PutObject operation (reached max retries: 4): Please reduce your request rate.